In [5]:
!pip install streamlit pyngrok


In [6]:
%%writefile negotiation_app.py
import numpy as np
import pandas as pd
import streamlit as st
import matplotlib.pyplot as plt
from scipy.optimize import linprog
import seaborn as sns
import json
import re

class SalaryNegotiationAgent:
    def __init__(self, name, preferences, role, experience, style):
        self.name = name
        self.preferences = preferences
        self.role = role
        self.experience = experience
        self.style = style

    def calculate_strategy(self, employer_preferences):
        payoff_matrix = np.array([self.preferences, employer_preferences])
        return self.nash_equilibrium(payoff_matrix)

    def nash_equilibrium(self, payoff_matrix):
        n_actions = payoff_matrix.shape[1]
        c = [-1] * n_actions
        A = -np.eye(n_actions)
        b = [0] * n_actions
        result = linprog(c, A_ub=A, b_ub=b, bounds=[(0, 1)] * n_actions, method='highs')
        if result.success:
            return result.x / result.x.sum()
        else:
            raise ValueError("Nash Equilibrium computation failed.")

    def propose(self):
        if self.style == "aggressive":
            return f"I demand {self.preferences[0]} as my starting salary."
        elif self.style == "collaborative":
            return "Let us find a mutually beneficial salary package."
        else:
            return "I am flexible but would like a fair compensation."

class Employer:
    def __init__(self, name, pay_range, style):
        self.name = name
        self.pay_range = pay_range
        self.style = style

    def respond_to_proposal(self, proposal, candidate_preferences):
        # Extract salary from proposal
        salary_match = re.search(r"\d+", proposal)
        proposed_salary = int(salary_match.group()) if salary_match else candidate_preferences[0]

        # Employer's salary range
        min_salary = min(self.pay_range)
        max_salary = max(self.pay_range)

        # Provide response based on style and salary range
        if proposed_salary < min_salary:
          return "Your proposal is below our minimum range. We are happy to accept it."
        elif proposed_salary > max_salary:
            if self.style == "aggressive":
              return "This is beyond our budget. We cannot accommodate this proposal."
            elif self.style == "collaborative":
              return "Your proposal is above our range, but let us find a compromise."
            else:  # Neutral style
              return "Your proposal exceeds our range, but we accept to foster goodwill."
        else:
          return "We accept your proposal as it aligns with our range."

st.set_page_config(page_title="Salary Negotiation System", layout="wide")
st.title("Salary Negotiation System")

if "candidates" not in st.session_state:
    st.session_state.candidates = []

if "employer" not in st.session_state:
    st.session_state.employer = None

if "negotiation_history" not in st.session_state:
    st.session_state.negotiation_history = []

# Candidate Input
st.sidebar.header("Add Candidate")
candidate_name = st.sidebar.text_input("Candidate Name")
role = st.sidebar.text_input("Role")
experience = st.sidebar.number_input("Years of Experience", min_value=0, max_value=50, step=1)
salary_preferences = st.sidebar.text_input("Salary Preferences (comma-separated)")
style = st.sidebar.selectbox("Negotiation Style", ["collaborative", "aggressive", "neutral"])

if st.sidebar.button("Add Candidate"):
    if candidate_name and role and salary_preferences:
        try:
            salary_preferences = list(map(int, salary_preferences.split(",")))
            st.session_state.candidates.append(SalaryNegotiationAgent(candidate_name, salary_preferences, role, experience, style))
            st.sidebar.success(f"Added Candidate: {candidate_name}")
        except ValueError:
            st.sidebar.error("Invalid salary preferences. Enter integers separated by commas.")
    else:
        st.sidebar.error("All fields are required!")

if st.sidebar.button("Save Candidates to JSON"):
    if st.session_state.candidates:
        candidates_data = [
            {
                "name": candidate.name,
                "preferences": candidate.preferences,
                "role": candidate.role,
                "experience": candidate.experience,
                "style": candidate.style
            }
            for candidate in st.session_state.candidates
        ]
        json_data = json.dumps(candidates_data, indent=4)
        st.sidebar.download_button(
            label="Download Candidates JSON",
            data=json_data,
            file_name="candidates.json",
            mime="application/json"
        )
    else:
        st.sidebar.error("No candidates to save.")

# Employer Input
st.sidebar.header("Add Employer")
employer_name = st.sidebar.text_input("Employer Name")
pay_range = st.sidebar.text_input("Pay Range (comma-separated)")
employer_style = st.sidebar.selectbox("Employer Negotiation Style", ["collaborative", "aggressive", "neutral"])

if st.sidebar.button("Add Employer"):
    if employer_name and pay_range:
        try:
            pay_range = list(map(int, pay_range.split(",")))
            st.session_state.employer = Employer(employer_name, pay_range, employer_style)
            st.sidebar.success(f"Added Employer: {employer_name}")
        except ValueError:
            st.sidebar.error("Invalid pay range. Enter integers separated by commas.")
    else:
        st.sidebar.error("All fields are required!")

# Display Participants
st.subheader("Current Participants")
if st.session_state.candidates:
    st.write("**Candidates:**")
    for idx, candidate in enumerate(st.session_state.candidates):
        st.write(f"**{candidate.name}**: Role: {candidate.role}, Experience: {candidate.experience} years, "
                 f"Salary Preferences: {candidate.preferences}, Style: {candidate.style}")
        if st.button(f"Remove Candidate {candidate.name}", key=f"remove_candidate_{idx}"):
            st.session_state.candidates.pop(idx)
            st.experimental_rerun()
else:
    st.write("No candidates added.")

if st.session_state.employer:
    st.write("**Employer:**")
    st.write(f"**{st.session_state.employer.name}**: Pay Range: {st.session_state.employer.pay_range}, "
             f"Style: {st.session_state.employer.style}")
    if st.button("Remove Employer"):
        st.session_state.employer = None
        st.experimental_rerun()
else:
    st.write("No employer added.")

# Simulate Negotiation
if st.button("Simulate Salary Negotiation"):
    if not st.session_state.candidates or not st.session_state.employer:
        st.error("Add at least one candidate and one employer to simulate!")
    else:
        st.subheader("Negotiation Results")
        results = []
        employer = st.session_state.employer

        for candidate in st.session_state.candidates:
            strategy = candidate.calculate_strategy(employer.pay_range)
            proposal = candidate.propose()
            response = employer.respond_to_proposal(proposal, candidate.preferences)

            result = {
                "Candidate": candidate.name,
                "Role": candidate.role,
                "Experience": candidate.experience,
                "Salary Preferences": candidate.preferences,
                "Proposal": proposal,
                "Response": response
            }
            results.append(result)
            st.session_state.negotiation_history.append(result)

        results_df = pd.DataFrame(results)
        st.dataframe(results_df)

        st.subheader("Visualization: Strategy Heatmap")
        strategy_matrix = pd.DataFrame(
            [row["Salary Preferences"] for row in results],
            columns=[f"Option {i+1}" for i in range(len(results[0]["Salary Preferences"]))],
            index=[row["Candidate"] for row in results]
        )

        fig_heatmap, ax_heatmap = plt.subplots(figsize=(10, 6))
        sns.heatmap(strategy_matrix, annot=True, fmt=".2f", cmap="coolwarm", ax=ax_heatmap)
        ax_heatmap.set_title("Candidate Salary Strategy Heatmap", fontsize=16)
        st.pyplot(fig_heatmap)


# View and Clear History
if st.button("View Negotiation History"):
    if st.session_state.negotiation_history:
        history_df = pd.DataFrame(st.session_state.negotiation_history)
        st.subheader("Negotiation History")
        st.dataframe(history_df)
    else:
        st.write("No negotiation history available.")

if st.sidebar.button("Clear Negotiation History"):
    st.session_state.negotiation_history = []
    st.sidebar.success("Negotiation history cleared.")

if st.sidebar.button("Clear All"):
    st.session_state.candidates = []
    st.session_state.employer = None
    st.session_state.negotiation_history = []
    st.sidebar.success("All data cleared.")



Overwriting negotiation_app.py


In [9]:
!ngrok config add-authtoken 2qAg8HG1NkifGidgpzMZXZhW5VG_5RQPHVLzcaqoP3jETrcdn


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [10]:
from pyngrok import ngrok
!streamlit run negotiation_app.py &>/dev/null&
# Specify the full address including the protocol
public_url = ngrok.connect(addr="http://localhost:8501")
print(f"Streamlit app is live at: {public_url}")

Streamlit app is live at: NgrokTunnel: "https://2893-35-201-243-42.ngrok-free.app" -> "http://localhost:8501"
